In [ ]:
import jax.numpy as jnp
import numpy as np
import time
import chex
import jax
import gymnasium as gym
import exciting_environments as excenv
import random
from exciting_environments import PMSM
import matplotlib.pyplot as plt
import diffrax
from jax.tree_util import tree_flatten, tree_unflatten, tree_structure

In [ ]:
BATCH_SIZE=5
new_motor_env = PMSM(
    saturated=True,
    LUT_motor_name="BRUSA",
    batch_size=BATCH_SIZE,
    control_state=["i_d","i_q"],#
    static_params = {
            "p": 3,
            "r_s": 17.932e-3,
            "l_d": 0.37e-3,
            "l_q": 1.2e-3,
            "psi_p": 65.65e-3,
            "deadtime": 0,
        })

In [ ]:
acts=jnp.full((5,8),jnp.array([1,1,1,1,1,1,1,1])*0.03)
acts=jnp.stack([acts,acts],axis=2)

In [ ]:
a= jnp.array([1,2,3])
a.repeat(0)

In [ ]:
state=new_motor_env.vmap_init_state()
for _ in range(acts.shape[1]):
    act=jnp.repeat(jnp.array([0.03,0.03])[:,None],BATCH_SIZE,axis=1).T
    obs,state=new_motor_env.vmap_step(state=state,action=act)
    print(obs[0,:])
    state_l=state

In [ ]:
state=new_motor_env.vmap_init_state()
obs,states,last_state=new_motor_env.vmap_sim_ahead(state,
                   acts,
                   new_motor_env.tau,
                   new_motor_env.tau
                   )
obs[0]

- add desired reference in states to generate reward, truncated and terminated

In [ ]:
import jax_dataclasses as jdc
init_state=new_motor_env.vmap_init_state()
with jdc.copy_and_mutate(states, validate=False) as states_ref:
            states_ref.reference.i_d=jnp.full((5,9),init_state.physical_state.i_d.reshape(-1,1))
            states_ref.reference.i_q=jnp.full((5,9),init_state.physical_state.i_q.reshape(-1,1))


In [ ]:
new_motor_env.vmap_generate_rew_trunc_term_ahead(states_ref,acts)